# Taxonomic Annotation of Reads

This notebook will go through the workflow assigning taxonomy to reads from a microbiome

Step 1: Assign taxonomy to reads (using 3 parts that run in a single job)
1. Taxonomic assignment of reads using Kraken2
2. Refinement of the taxonomic annotation using Bracken
3. Separation of human and non-human reads using KrakenTools

Step 2: Assign taxonomy to contigs from megahit (with 3 parts as above)

Step 3: Assign taxonomy to contigs from metaspades (with 3 parts as above)

## Getting Started

You will need to rerun this section each time you come back to this notebook to reset all directories and variables.

In [ ]:
# set the variables for your netid and xfile
netid = "MY_NETID"
xfile = "MY_XFILE"

In [ ]:
# Go into the working directory
work_dir = "/xdisk/bhurwitz/bh_class/" + netid + "/assignments/10_taxonomy"
%cd $work_dir

In [ ]:
# Set the fastq directory. This is where we have our fastq files with human contam removed.
fastq_dir = "/xdisk/bhurwitz/bh_class/" + netid + "/assignments/07_contam_removal"
xfile_dir = "/xdisk/bhurwitz/bh_class/" + netid + "/assignments/05_getting_data"

## Creating a config file
Let's create a config file with all of the variables we will need in the scripts below. Then when we want to use these variables in the script, we will "source" the config file to set the variables.

In [ ]:
# create a config file with all of the variables you need
# notice that now we are defining the tools here too, to make out scripts easier to read below
# also, if we want to update to a newer version of the tool we can just edit here.
!echo "export NETID=$netid" > config.sh
!echo "export XFILE=$xfile" >> config.sh
!echo "export WORK_DIR=$work_dir" >> config.sh
!echo "export XFILE_DIR=$xfile_dir" >> config.sh
!echo "export FASTQ_DIR=$fastq_dir" >> config.sh
!echo "export MEGAHIT_CONTIG_DIR=/xdisk/bhurwitz/bh_class/$netid/assignments/08_assembly/out_megahit" >> config.sh
!echo "export KRAKEN2=/contrib/singularity/shared/bhurwitz/kraken2:2.1.3--pl5321hdcf5f25_0.sif" >> config.sh
!echo "export BRACKEN=/contrib/singularity/shared/bhurwitz/bracken:2.8--py39h1f90b4d_1.sif" >> config.sh
!echo "export KRAKENTOOLS=/contrib/singularity/shared/bhurwitz/krakentools:1.2--pyh5e36f6f_0.sif" >> config.sh
!echo "export DB_DIR=/groups/bhurwitz/databases/kraken2/k2_pluspf_20230605" >> config.sh
!echo "export KMER_SIZE=100" >> config.sh


## Step 1: Assigning taxonomy to the reads

In this step, we will assign taxonomy to each of the reads in our microbiome (where possible). This script involves multiple steps in the annotation process including:

1. Running Kraken2 to assign reads to organisms by taxonomic rank. 
2. Running Bracken a tool that refines the Kraken2 output to try to reassign reads to higher species-level taxonomy
3. Separating out the reads based on their taxonomy using KrakenTools. We do this at a high level human / non-human.

The final part in this analysis looks for any remaining human contamination in the files, after we align to the human genome using bowtie2. Do you see any reads reported as human? Try "Blasting" these to see if they are indeed human reads that we missed by the alignment. 



In [ ]:
# Create a script to run Kraken, Bracken, and KrakenTools to assign taxonomy at the read-level.
# A few important points:
# 1. We are using the variables from the config file via the `source ./config.sh` command in the script.
# 2. Kraken2 runs on each of the fastq files in the trimmed/human screened $FASTQ_DIR
# 3. The results will be written into our $WORK_DIR
# 4. Notice that we are asking for alot more resource (24 cores and 5G of memory per core), we are also asking for 10 hours of runtime.
# But, runtime will likely be much less.
my_code = '''#!/bin/bash
#SBATCH --ntasks=1
#SBATCH --nodes=1             
#SBATCH --time=10:00:00   
#SBATCH --partition=standard
#SBATCH --account=bh_class
#SBATCH --array=0-7                         
#SBATCH --output=10A_read_taxonomy-%a.out
#SBATCH --cpus-per-task=24
#SBATCH --mem-per-cpu=5G  

pwd; hostname; date

source $SLURM_SUBMIT_DIR/config.sh
names=($(cat $XFILE_DIR/$XFILE))

SAMPLE_ID=${names[${SLURM_ARRAY_TASK_ID}]}

### reads with human removed to match to the reference database
PAIR1=${FASTQ_DIR}/${SAMPLE_ID}_1.fastq.gz
PAIR2=${FASTQ_DIR}/${SAMPLE_ID}_2.fastq.gz

KRAKEN_OUTDIR=${WORK_DIR}/out_reads_taxonomy
OUTDIR=${KRAKEN_OUTDIR}/${SAMPLE_ID}
HUMAN_READ_DIR=${OUTDIR}/human_reads
NONHUMAN_READ_DIR=${OUTDIR}/nonhuman_reads

### create the outdir if it does not exist
if [[ ! -d "$KRAKEN_OUTDIR" ]]; then
  echo "$KRAKEN_OUTDIR does not exist. Directory created"
  mkdir $KRAKEN_OUTDIR
fi

if [[ ! -d "$OUTDIR" ]]; then
  echo "$OUTDIR does not exist. Directory created"
  mkdir $OUTDIR
fi

if [[ ! -d "$HUMAN_READ_DIR" ]]; then
  echo "$HUMAN_READ_DIR does not exist. Directory created"
  mkdir $HUMAN_READ_DIR
fi

if [[ ! -d "$NONHUMAN_READ_DIR" ]]; then
  echo "$NONHUMAN_READ_DIR does not exist. Directory created"
  mkdir $NONHUMAN_READ_DIR
fi

# check input
echo ${PAIR1}
echo ${PAIR2}
echo ${OUTDIR}

apptainer run ${KRAKEN2} kraken2 --db ${DB_DIR} --paired \
  --classified-out ${OUTDIR}/cseqs#.fq --output ${OUTDIR}/kraken_results.txt \
  --report ${OUTDIR}/kraken_report.txt --use-names --threads ${SLURM_CPUS_PER_TASK} \
  ${PAIR1} ${PAIR2}

# refine hits with Bracken
REPORT="${OUTDIR}/kraken_report.txt"
RESULTS="${OUTDIR}/kraken_results.txt"
apptainer run ${BRACKEN} est_abundance.py -i ${REPORT} -o ${OUTDIR}/bracken_results.txt -k ${DB_DIR}/database${KMER_SIZE}mers.kmer_distrib

# get human and non-human reads (microbial)
TAXID=9606
HUMAN_R1="${HUMAN_READ_DIR}/r1.fq"
HUMAN_R2="${HUMAN_READ_DIR}/r2.fq"

BRACKEN_REPORT="${OUTDIR}/kraken_report_bracken_species.txt"
BRACKEN_RESULTS="${OUTDIR}/bracken_results.txt"

apptainer run ${KRAKENTOOLS} extract_kraken_reads.py -k ${RESULTS} \
 -r ${BRACKEN_REPORT} -s1 ${PAIR1} -s2 ${PAIR2} --taxid ${TAXID} \
 -o ${HUMAN_R1} -o2 ${HUMAN_R2} --include-children --fastq-output 

gzip ${HUMAN_READ_DIR}/r1.fq
gzip ${HUMAN_READ_DIR}/r2.fq

### selects all reads NOT from a given set of Kraken taxids (and all children)

NONHUMAN_R1="${NONHUMAN_READ_DIR}/r1.fq"
NONHUMAN_R2="${NONHUMAN_READ_DIR}/r2.fq"

apptainer run ${KRAKENTOOLS} extract_kraken_reads.py -k ${RESULTS} \
 -r ${BRACKEN_REPORT} -s1 ${PAIR1} -s2 ${PAIR2} --taxid ${TAXID} \
 -o ${NONHUMAN_R1} -o2 ${NONHUMAN_R2} --include-children \
 --exclude --fastq-output 

gzip ${NONHUMAN_READ_DIR}/r1.fq
gzip ${NONHUMAN_READ_DIR}/r2.fq

echo "Finished `date`"

'''

with open('10A_read_taxonomy.sh', mode='w') as file:
    file.write(my_code)

Nice, you should now have all of the reads assigned to the organisms they are from.

## Step 2: Assigning taxonomy to the contigs from Megahit

In this step, we will assign taxonomy to each of the contigs from our megahit assembly (where possible). This script involves multiple steps in the annotation process (just as before for the reads) including:

1. Running Kraken2 to assign contigs to organisms by taxonomic rank. 
2. Running Bracken a tool that refines the Kraken2 output to try to reassign contigs to higher species-level taxonomy
3. Separating out the contigs based on their taxonomy using KrakenTools. We do this at a high level human / non-human.

In [ ]:
# Create a script to run Kraken, Bracken, and KrakenTools to assign taxonomy at the contig-level for megahit.
# A few important points:
# 1. We are using the variables from the config file via the `source ./config.sh` command in the script.
# 2. Kraken2 runs on the final.contigs.fa file in the contig directory
# 3. The results will be written into our $WORK_DIR
# 4. Notice that we are asking for alot more resource (24 cores and 5G of memory per core), we are also asking for 10 hours of runtime.
# But, runtime will likely be much less.
my_code = '''#!/bin/bash
#SBATCH --ntasks=1
#SBATCH --nodes=1             
#SBATCH --time=10:00:00   
#SBATCH --partition=standard
#SBATCH --account=bh_class
#SBATCH --array=0-7                        
#SBATCH --output=10B_contig_taxonomy-%a.out
#SBATCH --cpus-per-task=24
#SBATCH --mem-per-cpu=5G  

pwd; hostname; date

source $SLURM_SUBMIT_DIR/config.sh
names=($(cat $XFILE_DIR/$XFILE))

SAMPLE_ID=${names[${SLURM_ARRAY_TASK_ID}]}

### contigs
CONTIGS=${MEGAHIT_CONTIG_DIR}/${SAMPLE_ID}/final.contigs.fa

KRAKEN_OUTDIR=${WORK_DIR}/out_megahit_taxonomy
OUTDIR=${KRAKEN_OUTDIR}/${SAMPLE_ID}
HUMAN_READ_DIR=${OUTDIR}/human_contigs
NONHUMAN_READ_DIR=${OUTDIR}/nonhuman_contigs

### create the outdir if it does not exist
if [[ ! -d "$KRAKEN_OUTDIR" ]]; then
  echo "$KRAKEN_OUTDIR does not exist. Directory created"
  mkdir $KRAKEN_OUTDIR
fi
                                  
if [[ ! -d "$OUTDIR" ]]; then
  echo "$OUTDIR does not exist. Directory created"
  mkdir $OUTDIR
fi

if [[ ! -d "$HUMAN_READ_DIR" ]]; then
  echo "$HUMAN_READ_DIR does not exist. Directory created"
  mkdir $HUMAN_READ_DIR
fi

if [[ ! -d "$NONHUMAN_READ_DIR" ]]; then
  echo "$NONHUMAN_READ_DIR does not exist. Directory created"
  mkdir $NONHUMAN_READ_DIR
fi

apptainer run ${KRAKEN2} kraken2 --db ${DB_DIR} --classified-out \
 ${OUTDIR}/cseqs#.fa --output ${OUTDIR}/kraken_results.txt \
 --report ${OUTDIR}/kraken_report.txt --use-names --threads ${SLURM_CPUS_PER_TASK} \
 ${CONTIGS}

# refine hits with Bracken
REPORT="${OUTDIR}/kraken_report.txt"
RESULTS="${OUTDIR}/kraken_results.txt"
apptainer run ${BRACKEN} est_abundance.py -i ${REPORT} -o ${OUTDIR}/bracken_results.txt -k ${DB_DIR}/database${KMER_SIZE}mers.kmer_distrib

# get human and non-human reads (microbial)
TAXID=9606
HUMAN_R1="${HUMAN_READ_DIR}/contigs.fa"

BRACKEN_REPORT="${OUTDIR}/kraken_report_bracken_species.txt"
BRACKEN_RESULTS="${OUTDIR}/bracken_results.txt"

apptainer run ${KRAKENTOOLS} extract_kraken_reads.py -k ${RESULTS} \
 -r ${BRACKEN_REPORT} -s1 ${CONTIGS} --taxid ${TAXID} -o ${HUMAN_R1} \
 --include-children 

if [[ -f "${HUMAN_READ_DIR}/contigs.fa" ]]; then
    gzip ${HUMAN_READ_DIR}/contigs.fa
fi

### selects all reads NOT from a given set of Kraken taxids (and all children)

NONHUMAN_R1="${NONHUMAN_READ_DIR}/contigs.fa"

apptainer run ${KRAKENTOOLS} extract_kraken_reads.py -k ${RESULTS} \
 -r ${BRACKEN_REPORT} -s1 ${CONTIGS} --taxid ${TAXID} -o ${NONHUMAN_R1} \
 --include-children --exclude 

if [[ -f "${NONHUMAN_READ_DIR}/contigs.fa" ]]; then
    gzip ${NONHUMAN_READ_DIR}/contigs.fa
fi

echo "Finished `date`"

'''

with open('10B_contig_taxonomy.sh', mode='w') as file:
    file.write(my_code)

## Step 3: Putting it all together

Once you have created the the run scripts, you are ready to put them together in a pipeline to run each of the steps one by one. Notice which steps are dependent on the others.

In [ ]:
# Let's create the launcher script to kick off our pipeline.

my_code = '''#! /bin/bash

# 10A_read_taxonomy: jid1 has no dependencies
job1=$(sbatch 10A_read_taxonomy.sh)
jid1=$(echo $job1 | sed 's/^Submitted batch job //')
echo $jid1

# 10B_contig_taxonomy: jid2 has no dependencies
job2=$(sbatch 10B_contig_taxonomy.sh)
jid2=$(echo $job2 | sed 's/^Submitted batch job //')
echo $jid2

'''

with open('10_launch_pipeline.sh', mode='w') as file:
    file.write(my_code)

In [ ]:
# Make the pipeline script executable
!chmod +x *.sh

In [ ]:
# now let's run it!
!./10_launch_pipeline.sh

In [ ]:
# You can check if it is running using the squeue command
# Check for all jobs under your netid
# Note that this will take some time to run, so go get a coffee!
!squeue --user=$netid

## Final Step
Copy your notebook to the current working directory

In [ ]:
!cp ~/be487-fall-2024/assignments/10_taxonomy/hw10_taxonomy.ipynb $work_dir